# Baseline Gender Biased Token Classifiers

### Target: Label Categories

### Word Embeddings: GloVe

* Supervised learning
    * Train, Validate, and (Blind) Test Data: under directory `../data/token_clf_data/model_input/`
    * Prediction Data: under directory `../data/token_clf_data/model_output`
* Multilabel classification
    * 3 categories of labels: Linguistic, Person Name, Contextual
* Word Embeddings
    * GloVe (trained on Common Crawl + English Wikipedia dump)

***

### Table of Contents

**[0.](#0) Preprocessing**

**[1.](#1) Models**

**[2.](#2) Performance Evaluation**

***

Load necessary libraries:

In [1]:
# For custom functions and variables
import utils, config

# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For visualization
import matplotlib.pyplot as plt

# For preprocessing
from nltk.stem import WordNetLemmatizer
import scipy.stats

# For classification
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

# For evaluation
from collections import Counter
from sklearn.metrics import classification_report, make_scorer
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, ConfusionMatrixDisplay #, plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, f1_score
from intervaltree import Interval, IntervalTree

<a id="0"></a>
## 0. Preprocessing

Load the train and validation (dev) data:

In [2]:
df_train = pd.read_csv(config.tokc_path+"model_input/token_train.csv", index_col=0)
df_dev = pd.read_csv(config.tokc_path+"model_input/token_validate.csv", index_col=0)
print(df_train.shape, df_dev.shape)
df_train.head()

(467564, 10) (157740, 10)


,description_id,sentence_id,ann_id,token_id,token,token_offsets,pos,tag,field,subset
3,1,1,99999,3,Title,"(17, 22)",NN,O,Title,train
4,1,1,99999,4,:,"(22, 23)",:,O,Title,train
5,1,1,99999,5,Papers,"(24, 30)",NNS,O,Title,train
6,1,1,99999,6,of,"(31, 33)",IN,O,Title,train
7,1,1,14384,7,The,"(34, 37)",DT,B-Unknown,Title,train


Drop duplicate rows with all but the same annotation ID:

In [3]:
df_train = df_train.drop(columns=["ann_id"])
df_train = df_train.drop_duplicates()
df_dev = df_dev.drop(columns=["ann_id"])
df_dev = df_dev.drop_duplicates()
print(df_train.shape, df_dev.shape)

(463441, 9) (156146, 9)


Remove Non-binary labels as these were mistaken labels identified early on that were meant to be excluded, and because only one token has this label, it prevents the data from being input into the models with cross-validation.

In [4]:
df_train = df_train.loc[df_train.tag != "B-Nonbinary"]
df_train = df_train.loc[df_train.tag != "I-Nonbinary"]

In [5]:
df_train.shape

(463439, 9)

***

#### Label Categories

Add the annotation label categories as a column of higher-level Inside-Outside-Beginning (IOB) tags so they can be used as targets:

In [6]:
df_train = utils.addCategoryTagColumn(df_train)
# df_train.head(20)

In [7]:
df_dev = utils.addCategoryTagColumn(df_dev)
df_dev.head()

,description_id,sentence_id,token_id,token,token_offsets,pos,tag,field,subset,tag_cat
172,3,5,154,After,"(907, 912)",IN,O,Biographical / Historical,dev,O
173,3,5,155,his,"(913, 916)",PRP$,B-Gendered-Pronoun,Biographical / Historical,dev,B-Linguistic
174,3,5,156,ordination,"(917, 927)",NN,O,Biographical / Historical,dev,O
175,3,5,157,he,"(928, 930)",PRP,B-Gendered-Pronoun,Biographical / Historical,dev,B-Linguistic
176,3,5,158,spent,"(931, 936)",VBD,O,Biographical / Historical,dev,O


Remove columns that won't be used as features for the classifiers and remove any duplicate rows that remain:

In [8]:
cols_to_keep = ["sentence_id", "token_id", "pos", "token", "tag_cat"]

In [9]:
df_train = df_train[cols_to_keep]
df_train = df_train.drop_duplicates()
df_dev = df_dev[cols_to_keep]
df_dev = df_dev.drop_duplicates()
# df_train.head(20)

Create columns for each category so they can be passed into the models as individual features:

In [10]:
ling_cat_tags = ["B-Linguistic", "I-Linguistic"]
df_train_ling = df_train.loc[df_train.tag_cat.isin(ling_cat_tags)]
df_dev_ling = df_dev.loc[df_dev.tag_cat.isin(ling_cat_tags)]

In [11]:
pers_cat_tags = ["B-Person-Name", "I-Person-Name"]
df_train_pers = df_train.loc[df_train.tag_cat.isin(pers_cat_tags)]
df_dev_pers = df_dev.loc[df_dev.tag_cat.isin(pers_cat_tags)]

In [12]:
cont_cat_tags = ["B-Contextual", "I-Contextual"]
df_train_cont = df_train.loc[df_train.tag_cat.isin(cont_cat_tags)]
df_dev_cont = df_dev.loc[df_dev.tag_cat.isin(cont_cat_tags)]

In [13]:
df_train = (df_train.drop(columns=["tag_cat"])).drop_duplicates()
df_dev = (df_dev.drop(columns=["tag_cat"])).drop_duplicates()

In [14]:
join_cols = ["sentence_id", "token_id", "pos", "token"]

In [15]:
df_train = df_train.join(df_train_ling.set_index(join_cols), on=join_cols, how="outer")
df_train = df_train.join(df_train_pers.set_index(join_cols), on=join_cols, how="outer", lsuffix="", rsuffix="_personname")
df_train = df_train.join(df_train_cont.set_index(join_cols), on=join_cols, how="outer", lsuffix="", rsuffix="_contextual")
df_train = df_train.rename(columns={"tag_cat":"tag_cat_linguistic"})
# df_train.head(30)  # Should have one row per token!

In [16]:
df_dev = df_dev.join(df_dev_ling.set_index(join_cols), on=join_cols, how="outer")
df_dev = df_dev.join(df_dev_pers.set_index(join_cols), on=join_cols, how="outer", lsuffix="", rsuffix="_personname")
df_dev = df_dev.join(df_dev_cont.set_index(join_cols), on=join_cols, how="outer", lsuffix="", rsuffix="_contextual")
df_dev = df_dev.rename(columns={"tag_cat":"tag_cat_linguistic"})
# df_dev.head(30)

Replace the `tag_cat_` columns' `nan` values with `'O'`:

In [17]:
tag_cat_cols = ["tag_cat_linguistic", "tag_cat_personname", "tag_cat_contextual"]
df_train[tag_cat_cols] = df_train[tag_cat_cols].fillna('O')
df_dev[tag_cat_cols] = df_dev[tag_cat_cols].fillna('O')
df_dev.head()

,sentence_id,token_id,pos,token,tag_cat_linguistic,tag_cat_personname,tag_cat_contextual
172,5,154,IN,After,O,O,O
173,5,155,PRP$,his,B-Linguistic,O,O
174,5,156,NN,ordination,O,O,O
175,5,157,PRP,he,B-Linguistic,O,O
176,5,158,VBD,spent,O,O,O


Group the data by sentence, so the token column becomes a list of tokens for each sentence:

In [18]:
df_train_grouped = utils.implodeDataFrame(df_train, ["sentence_id"])
df_dev_grouped = utils.implodeDataFrame(df_dev, ["sentence_id"])
df_train_grouped = df_train_grouped.rename(columns={"token":"sentence"})
df_dev_grouped = df_dev_grouped.rename(columns={"token":"sentence"})
df_dev_grouped.head()

,token_id,pos,sentence,tag_cat_linguistic,tag_cat_personname,tag_cat_contextual
sentence_id,,,,,,
5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...","[IN, PRP$, NN, PRP, VBD, CD, NNS, IN, DT, NN, ...","[After, his, ordination, he, spent, three, yea...","[O, B-Linguistic, O, B-Linguistic, O, O, O, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, B-Contextual, I-Co..."
11,"[308, 309, 310]","[NN, :, NN]","[Identifier, :, AA6]","[O, O, O]","[O, O, O]","[O, O, O]"
13,"[321, 322, 323, 324, 325, 326, 327, 328, 329, ...","[NN, CC, NNS, :, NNS, CC, NNS, ,, JJ, ;, NNS, ...","[Scope, and, Contents, :, Sermons, and, addres...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
18,"[498, 499, 500, 501, 502, 503, 504, 505, 506, ...","[IN, CD, NNP, NNP, VBD, NNP, NNP, CC, PRP, VBD...","[In, 1941, Tom, Allan, married, Jane, Moore, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, B-Person-Name, I-Person-Name, O, B-Pers...","[O, O, B-Contextual, I-Contextual, I-Contextua..."
24,"[649, 650, 651, 652, 653, 654, 655, 656, 657, ...","[IN, CD, NNP, NNP, NNP, VBD, DT, NN, TO, VB, N...","[In, 1955, Rev, Tom, Allan, accepted, a, call,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, B-Person-Name, I-Person-Name, I-Person-...","[O, O, O, O, O, O, O, O, O, O, B-Contextual, O..."


Zip the POS and category tags together with the tokens so each sentence item is a tuple: `(TOKEN, POS-TAG, CATEGORY-TAG)`

In [19]:
train_sentences_ling = utils.zipFeaturesAndTarget(df_train_grouped, "tag_cat_linguistic")
print(train_sentences_ling[0][:3])
dev_sentences_ling = utils.zipFeaturesAndTarget(df_dev_grouped, "tag_cat_linguistic")
print(dev_sentences_ling[0][:3])
train_sentences_pers = utils.zipFeaturesAndTarget(df_train_grouped, "tag_cat_personname")
dev_sentences_pers = utils.zipFeaturesAndTarget(df_dev_grouped, "tag_cat_personname")
train_sentences_cont = utils.zipFeaturesAndTarget(df_train_grouped, "tag_cat_contextual")
dev_sentences_cont = utils.zipFeaturesAndTarget(df_dev_grouped, "tag_cat_contextual")

[('Title', 'NN', 'O'), (':', ':', 'O'), ('Papers', 'NNS', 'O')]
[('After', 'IN', 'O'), ('his', 'PRP$', 'B-Linguistic'), ('ordination', 'NN', 'O')]


#### Word Embeddings

Get GloVe word embeddings (which were trained on English Wikipedia entries) for the vocabulary of the dataset (the unique tokens in the training set):

In [20]:
dimensions = ["50", "100", "200", "300"]
d = dimensions[0]  
# dimensonality of custom fastText embeddings = 100

In [21]:
glove = utils.getGloveEmbeddings(d)
# print(glove["the"])

In [22]:
vocabulary = list(df_train.token.unique())
vocabulary_lowercased = [token.lower() for token in vocabulary]
vocabulary_lowercased = list(set(vocabulary_lowercased))
print("Vocabulary size:", len(vocabulary))
print("Lowercased vocabulary size:", len(vocabulary_lowercased))

Vocabulary size: 35968
Lowercased vocabulary size: 31335


In [23]:
word_embeddings = utils.getEmbeddingsForTokens(glove, vocabulary)

In [24]:
assert np.array_equal(word_embeddings[0], glove[vocabulary[0].lower()])

In [25]:
embedding_dict = dict(zip(vocabulary, word_embeddings))

In [26]:
embedding_dict_keys = list(embedding_dict.keys())
for token in vocabulary:
    assert token in embedding_dict_keys

Create feature dictionaries:

*References:*
* *https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html*
* *https://stackoverflow.com/questions/58736548/how-to-use-word-embedding-as-features-for-crf-sklearn-crfsuite-model-training*

In [27]:
# Get a vector representation of a token from a dictionray of word embeddings
def extractEmbedding(token, embedding_dict=glove, dimensions=int(d)):
    if token.isalpha():
        token = token.lower()
    try:
        embedding = embedding_dict[token]
    except KeyError:
        embedding = np.zeros((dimensions,))
    return embedding.reshape(-1,1)

def extractTokenFeatures(sentence, i):
    token = sentence[i][0]
    pos = sentence[i][1]
    features = {
        'bias': 1.0,
        'pos': pos,
        'pos[:2]': pos[:2],
        'token': token
    }
    
    # Add each value in a token's word embedding as a separate feature
    embedding = extractEmbedding(token)
    for i,n in enumerate(embedding):
        features['e{}'.format(i)] = n
    
    # Record whether a token is the first or last token of a sentence
    if i == 0:
        features['START'] = True
    elif i == (len(sentence) - 1):
        features['END'] = True
    
    return features

def extractSentenceFeatures(sentence):
    return [extractTokenFeatures(sentence, i) for i in range(len(sentence))]

def extractSentenceTargets(sentence):
    return [tag for token, pos, tag in sentence]

def extractSentenceTokens(sentence):
    return [token for token, pos, tag in sentence]

In [28]:
# extractSentenceFeatures(train_sentences[0])[5]

<a id="1"></a>
## 1. Models

### Linguistic

* **Features:** part-of-speech tag, first 2 letters of part-of-speech tag abbreviation, GloVe embeddings
* **Target:** Linguistic label category IOB tags
* **Algorithm:** L2SGD

#### Train

Train a Conditional Random Field (CRF) model with the default parameters on the **Linguistic** category of tags:

In [29]:
train_sentences = train_sentences_ling
dev_sentences = dev_sentences_ling

In [30]:
# Features
X_train = [extractSentenceFeatures(sentence) for sentence in train_sentences]
X_dev = [extractSentenceFeatures(sentence) for sentence in dev_sentences]
# Target
y_train = [extractSentenceTargets(sentence) for sentence in train_sentences]
y_dev = [extractSentenceTargets(sentence) for sentence in dev_sentences]

In [31]:
algorithms = ['lbfgs', 'l2sgd', 'ap', 'pa', 'arow']
# Available algorithms with sklearn_crfsuite are:
#     'lbfgs' - Gradient descent using the L-BFGS method
#     'l2sgd' - Stochastic Gradient Descent with L2 regularization term
#     'ap' - Averaged Perceptron
#     'pa' - Passive Aggressive (PA)
#     'arow' - Adaptive Regularization Of Weight Vector (AROW)

In [32]:
# clf = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100) #iterations unlimited
clf = sklearn_crfsuite.CRF(algorithm=algorithms[1], c2=0.1, max_iterations=100, all_possible_transitions=True)     # up to 1000 iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[2], max_iterations=100)           # max iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[3], max_iterations=100)           # max iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[4], max_iterations=100)           # max iterations allowed

In [33]:
# https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles
try:
    clf.fit(X_train, y_train)
except AttributeError:
    pass

Remove `'O'` tags from the targets list since we are interested in the ability to apply the gendered and gender biased language related tags, and the `'O'` tags far outnumber the tags for gendered and gender biased language.

In [34]:
targets = list(clf.classes_)
targets.remove('O')
print(targets)

['B-Linguistic', 'I-Linguistic']


#### Predict

In [35]:
y_pred = clf.predict(X_dev)

#### Evaluate

##### Strict Evaluation

In [36]:
print("  - F1:", metrics.flat_f1_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Prec:", metrics.flat_precision_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Rec", metrics.flat_recall_score(y_dev, y_pred, average="weighted", labels=targets))

  - F1: 0.7486667314705173
  - Prec: 0.795437655165426
  - Rec 0.735140771637122


In [37]:
# targets_sorted = sorted(targets, key=lambda name: (name[1:], name[0]))
# print(metrics.flat_classification_report(y_dev, y_pred, labels=targets_sorted, digits=5))

Experiments: comparison of all available algorithms:

In [38]:
# # LBFGS - third-best
# targets_sorted = sorted(targets, key=lambda name: (name[1:], name[0]))
# print(metrics.flat_classification_report(y_dev, y_pred, labels=targets_sorted, digits=5))

In [39]:
# # L2SGD - tied for first (about the same as the fourth)
# targets_sorted = sorted(targets, key=lambda name: (name[1:], name[0]))
# print(metrics.flat_classification_report(y_dev, y_pred, labels=targets_sorted, digits=5))

In [40]:
# # AP - second-best
# targets_sorted = sorted(targets, key=lambda name: (name[1:], name[0]))
# print(metrics.flat_classification_report(y_dev, y_pred, labels=targets_sorted, digits=5))

In [41]:
# # PA - tied for first (about the same as the second)
# targets_sorted = sorted(targets, key=lambda name: (name[1:], name[0]))
# print(metrics.flat_classification_report(y_dev, y_pred, labels=targets_sorted, digits=5))

In [42]:
# # AROW - worst-performing
# targets_sorted = sorted(targets, key=lambda name: (name[1:], name[0]))
# print(metrics.flat_classification_report(y_dev, y_pred, labels=targets_sorted, digits=5))

Save the prediction data:

In [43]:
df_dev_grouped = df_dev_grouped.rename(columns={"tag_cat_linguistic":"tag_cat_linguistic_expected"})
df_dev_grouped.insert(len(df_dev_grouped.columns), "tag_cat_linguistic_predicted", y_pred)
# df_dev_grouped.head()

The built-in evaluation approach is strict, so unless the model predictions' labels are on text spans that exactly match the development data's test, the predicted labels will be deemed incorrect.

As with the manual annotation evaluation, we want to also evaluate the predictions more loosely, considering overlapping text spans in addition to exactly matching text spans.

### Person Name

* **Features:** part-of-speech tag, first 2 letters of part-of-speech tag abbreviation, custom fastText embeddings
* **Target:** Person-Name label category IOB tags
* **Algorithm:** L2SGD

#### Train

Train a Conditional Random Field (CRF) model with the default parameters on the **Person Name** category of tags:

In [44]:
train_sentences = train_sentences_pers
dev_sentences = dev_sentences_pers

In [45]:
# Features
X_train = [extractSentenceFeatures(sentence) for sentence in train_sentences]
X_dev = [extractSentenceFeatures(sentence) for sentence in dev_sentences]
# Target
y_train = [extractSentenceTargets(sentence) for sentence in train_sentences]
y_dev = [extractSentenceTargets(sentence) for sentence in dev_sentences]

In [46]:
algorithms = ['lbfgs', 'l2sgd', 'ap', 'pa', 'arow']

In [47]:
clf = sklearn_crfsuite.CRF(algorithm=algorithms[1], c2=0.1, max_iterations=100, all_possible_transitions=True)

In [48]:
# https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles
try:
    clf.fit(X_train, y_train)
except AttributeError:
    pass

Remove `'O'` tags from the targets list since we are interested in the ability to apply the gendered and gender biased language related tags, and the `'O'` tags far outnumber the tags for gendered and gender biased language.

In [49]:
targets = list(clf.classes_)
targets.remove('O')
print(targets)

['B-Person-Name', 'I-Person-Name']


#### Predict

In [50]:
y_pred = clf.predict(X_dev)

#### Evaluate

##### Strict Evaluation Summary

In [51]:
print("  - F1:", metrics.flat_f1_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Prec:", metrics.flat_precision_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Rec", metrics.flat_recall_score(y_dev, y_pred, average="weighted", labels=targets))

  - F1: 0.6836995226710204
  - Prec: 0.7664732820029956
  - Rec 0.6174724342663274


In [52]:
# targets_sorted = sorted(targets, key=lambda name: (name[1:], name[0]))
# print(metrics.flat_classification_report(y_dev, y_pred, labels=targets_sorted, digits=5))

Save the prediction data:

In [53]:
df_dev_grouped = df_dev_grouped.rename(columns={"tag_cat_personname":"tag_cat_personname_expected"})
df_dev_grouped.insert(len(df_dev_grouped.columns), "tag_cat_personname_predicted", y_pred)
# df_dev_grouped.head()

<a id="1"></a>
### Contextual

* **Features:** part-of-speech tag, first 2 letters of part-of-speech tag abbreviation, custom fastText embeddings
* **Target:** Contextual label category IOB tags
* **Algorithm:** L2SGD

#### Train

Train a Conditional Random Field (CRF) model with the default parameters on the **Contextual** category of tags:

In [54]:
train_sentences = train_sentences_cont
dev_sentences = dev_sentences_cont

In [55]:
# Features
X_train = [extractSentenceFeatures(sentence) for sentence in train_sentences]
X_dev = [extractSentenceFeatures(sentence) for sentence in dev_sentences]
# Target
y_train = [extractSentenceTargets(sentence) for sentence in train_sentences]
y_dev = [extractSentenceTargets(sentence) for sentence in dev_sentences]

In [56]:
algorithms = ['lbfgs', 'l2sgd', 'ap', 'pa', 'arow']

In [57]:
clf = sklearn_crfsuite.CRF(algorithm=algorithms[1], c2=0.1, max_iterations=100, all_possible_transitions=True)

In [58]:
# https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles
try:
    clf.fit(X_train, y_train)
except AttributeError:
    pass

Remove `'O'` tags from the targets list since we are interested in the ability to apply the gendered and gender biased language related tags, and the `'O'` tags far outnumber the tags for gendered and gender biased language.

In [59]:
targets = list(clf.classes_)
targets.remove('O')
print(targets)

['B-Contextual', 'I-Contextual']


#### Predict

In [60]:
y_pred = clf.predict(X_dev)

#### Evaluate

##### Strict Evaluation Summary

In [61]:
print("  - F1:", metrics.flat_f1_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Prec:", metrics.flat_precision_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Rec", metrics.flat_recall_score(y_dev, y_pred, average="weighted", labels=targets))

  - F1: 0.32005165248636236
  - Prec: 0.6812817844789257
  - Rec 0.20969419596421884


In [62]:
# targets_sorted = sorted(targets, key=lambda name: (name[1:], name[0]))
# print(metrics.flat_classification_report(y_dev, y_pred, labels=targets_sorted, digits=5))

Save the prediction data:

In [63]:
df_dev_grouped = df_dev_grouped.rename(columns={"tag_cat_contextual":"tag_cat_contextual_expected"})
df_dev_grouped.insert(len(df_dev_grouped.columns), "tag_cat_contextual_predicted", y_pred)
df_dev_grouped.head()

,token_id,pos,sentence,tag_cat_linguistic_expected,tag_cat_personname_expected,tag_cat_contextual_expected,tag_cat_linguistic_predicted,tag_cat_personname_predicted,tag_cat_contextual_predicted
sentence_id,,,,,,,,,
5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...","[IN, PRP$, NN, PRP, VBD, CD, NNS, IN, DT, NN, ...","[After, his, ordination, he, spent, three, yea...","[O, B-Linguistic, O, B-Linguistic, O, O, O, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, B-Contextual, I-Co...","[O, B-Linguistic, O, B-Linguistic, O, O, O, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
11,"[308, 309, 310]","[NN, :, NN]","[Identifier, :, AA6]","[O, O, O]","[O, O, O]","[O, O, O]","[O, O, O]","[O, O, O]","[O, O, O]"
13,"[321, 322, 323, 324, 325, 326, 327, 328, 329, ...","[NN, CC, NNS, :, NNS, CC, NNS, ,, JJ, ;, NNS, ...","[Scope, and, Contents, :, Sermons, and, addres...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
18,"[498, 499, 500, 501, 502, 503, 504, 505, 506, ...","[IN, CD, NNP, NNP, VBD, NNP, NNP, CC, PRP, VBD...","[In, 1941, Tom, Allan, married, Jane, Moore, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, B-Person-Name, I-Person-Name, O, B-Pers...","[O, O, B-Contextual, I-Contextual, I-Contextua...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, B-Person-Name, I-Person-Name, O, B-Pers...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
24,"[649, 650, 651, 652, 653, 654, 655, 656, 657, ...","[IN, CD, NNP, NNP, NNP, VBD, DT, NN, TO, VB, N...","[In, 1955, Rev, Tom, Allan, accepted, a, call,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, B-Person-Name, I-Person-Name, I-Person-...","[O, O, O, O, O, O, O, O, O, O, B-Contextual, O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, B-Person-Name, I-Person-Name, I-Person-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


<a id="2"></a>
## 2. Performance Evaluation

### Strict Evaluation

The built-in evaluation approach is strict, so unless the model predictions' labels are on text spans that exactly match the development data's test, the predicted labels will be deemed incorrect.

As with the manual annotation evaluation, we want to evaluate the predictions more loosely, considering overlapping text spans in addition to exactly matching text spans.  Save the predictions for each token and then use IntervalTree to evaluate performance considering overlapping labels, rather than only exactly matching labels.

In [64]:
df_dev_exploded = df_dev_grouped.explode(list(df_dev_grouped.columns))
df_dev_exploded = df_dev_exploded.rename(columns={"sentence":"token"})
df_dev_exploded.head()

,token_id,pos,token,tag_cat_linguistic_expected,tag_cat_personname_expected,tag_cat_contextual_expected,tag_cat_linguistic_predicted,tag_cat_personname_predicted,tag_cat_contextual_predicted
sentence_id,,,,,,,,,
5,154,IN,After,O,O,O,O,O,O
5,155,PRP$,his,B-Linguistic,O,O,B-Linguistic,O,O
5,156,NN,ordination,O,O,O,O,O,O
5,157,PRP,he,B-Linguistic,O,O,B-Linguistic,O,O
5,158,VBD,spent,O,O,O,O,O,O


In [65]:
print(df_dev_exploded.shape)
print(df_dev.shape)

(152711, 9)
(152711, 7)


Save grouped (one row per sentence) and exploded (one row per token) versions of the data:

In [66]:
df_dev_grouped.to_csv(config.tokc_path+"model_output/categoryTags_crfL2sgd_POS-GloVe50_bySentence.csv")
df_dev_exploded.to_csv(config.tokc_path+"model_output/categoryTags_crfL2sgd_POS-GloVe50_byToken.csv")
# df_dev_exploded = pd.read_csv(config.tokc_path+"model_output/categoryTags_crfL2sgd_POS-CustomFastText_byToken.csv")
# df_dev_exploded.head()

Strictly evaluate the **Linguistic** tags:

In [67]:
df_dev_exploded = df_dev_exploded.reset_index()

In [68]:
category = "linguistic"
tags = ["B-Linguistic", "I-Linguistic"]
subdf_pred_ling = utils.addCatAgreementAndMatchTypeCols(df_dev_exploded, category, tags)
subdf_pred_ling.head()

,sentence_id,token_id,token,tag_cat_linguistic_expected,tag_cat_linguistic_predicted,strict_agreement,match_type
0,5,154,After,O,O,TN,exact_match
1,5,155,his,B-Linguistic,B-Linguistic,TP,exact_match
2,5,156,ordination,O,O,TN,exact_match
3,5,157,he,B-Linguistic,B-Linguistic,TP,exact_match
4,5,158,spent,O,O,TN,exact_match


In [69]:
subdf_pred_ling_stats = subdf_pred_ling.groupby("strict_agreement").count()
subdf_pred_ling_stats = subdf_pred_ling_stats[["token_id"]]
subdf_pred_ling_stats = subdf_pred_ling_stats.rename(columns={"token_id":"linguistic_count"})
subdf_pred_ling_stats

,linguistic_count
strict_agreement,
FN,506
FP,351
TN,150444
TP,1410


In [70]:
subdf_pred_ling_stats2 = subdf_pred_ling.groupby("match_type").count()
subdf_pred_ling_stats2 = subdf_pred_ling_stats2[["token_id"]]
subdf_pred_ling_stats2 = subdf_pred_ling_stats2.rename(columns={"token_id":"linguistic_count"})
subdf_pred_ling_stats2

,linguistic_count
match_type,
category_match,2
exact_match,151854
mismatch,855


Strictly evaluate the **Person Name** tags:

In [71]:
category = "personname"
tags = ["B-Person-Name", "I-Person-Name"]
subdf_pred_pers = utils.addCatAgreementAndMatchTypeCols(df_dev_exploded, category, tags)
subdf_pred_pers.head()

,sentence_id,token_id,token,tag_cat_personname_expected,tag_cat_personname_predicted,strict_agreement,match_type
0,5,154,After,O,O,TN,exact_match
1,5,155,his,O,O,TN,exact_match
2,5,156,ordination,O,O,TN,exact_match
3,5,157,he,O,O,TN,exact_match
4,5,158,spent,O,O,TN,exact_match


In [72]:
subdf_pred_pers_stats = subdf_pred_pers.groupby("strict_agreement").count()
subdf_pred_pers_stats = subdf_pred_pers_stats[["token_id"]]
subdf_pred_pers_stats = subdf_pred_pers_stats.rename(columns={"token_id":"personname_count"})
subdf_pred_pers_stats

,personname_count
strict_agreement,
FN,2367
FP,1332
TN,144644
TP,4368


In [73]:
subdf_pred_pers_stats2 = subdf_pred_pers.groupby("match_type").count()
subdf_pred_pers_stats2 = subdf_pred_pers_stats2[["token_id"]]
subdf_pred_pers_stats2 = subdf_pred_pers_stats2.rename(columns={"token_id":"personname_count"})
subdf_pred_pers_stats2

,personname_count
match_type,
category_match,339
exact_match,149012
mismatch,3360


Strictly evaluate the **Contextual** tags:

In [74]:
category = "contextual"
tags = ["B-Contextual", "I-Contextual"]
subdf_pred_cont = utils.addCatAgreementAndMatchTypeCols(df_dev_exploded, category, tags)
subdf_pred_cont.head()

,sentence_id,token_id,token,tag_cat_contextual_expected,tag_cat_contextual_predicted,strict_agreement,match_type
0,5,154,After,O,O,TN,exact_match
1,5,155,his,O,O,TN,exact_match
2,5,156,ordination,O,O,TN,exact_match
3,5,157,he,O,O,TN,exact_match
4,5,158,spent,O,O,TN,exact_match


In [75]:
subdf_pred_cont_stats = subdf_pred_cont.groupby("strict_agreement").count()
subdf_pred_cont_stats = subdf_pred_cont_stats[["token_id"]]
subdf_pred_cont_stats = subdf_pred_cont_stats.rename(columns={"token_id":"contextual_count"})
subdf_pred_cont_stats

,contextual_count
strict_agreement,
FN,3667
FP,448
TN,147588
TP,1008


In [76]:
subdf_pred_cont_stats2 = subdf_pred_cont.groupby("match_type").count()
subdf_pred_cont_stats2 = subdf_pred_cont_stats2[["token_id"]]
subdf_pred_cont_stats2 = subdf_pred_cont_stats2.rename(columns={"token_id":"contextual_count"})
subdf_pred_cont_stats2

,contextual_count
match_type,
category_match,132
exact_match,148596
mismatch,3983


Combine the statistics and calculate precision, recall, and F1 scores for each label:

In [77]:
subdf_pred_stats = pd.concat([subdf_pred_ling_stats.T, subdf_pred_pers_stats.T, subdf_pred_cont_stats.T])
subdf_pred_stats

strict_agreement,FN,FP,TN,TP
linguistic_count,506,351,150444,1410
personname_count,2367,1332,144644,4368
contextual_count,3667,448,147588,1008


In [78]:
lprec, lrec, lf = utils.precisionRecallF1(subdf_pred_stats.TP.values[0], subdf_pred_stats.FP.values[0], subdf_pred_stats.FN.values[0])
pprec, prec, pf = utils.precisionRecallF1(subdf_pred_stats.TP.values[1], subdf_pred_stats.FP.values[1], subdf_pred_stats.FN.values[1])
cprec, crec, cf = utils.precisionRecallF1(subdf_pred_stats.TP.values[2], subdf_pred_stats.FP.values[2], subdf_pred_stats.FN.values[2])
precision = [lprec, pprec, cprec]
recall = [lrec, prec, crec]
f_1 = [lf, pf, cf]
subdf_pred_stats.insert(len(list(subdf_pred_stats.columns)), "precision", precision)
subdf_pred_stats.insert(len(list(subdf_pred_stats.columns)), "recall", recall)
subdf_pred_stats.insert(len(list(subdf_pred_stats.columns)), "f_1", f_1)
subdf_pred_stats

strict_agreement,FN,FP,TN,TP,precision,recall,f_1
linguistic_count,506,351,150444,1410,0.800681,0.735908,0.766930
personname_count,2367,1332,144644,4368,0.766316,0.648552,0.702533
contextual_count,3667,448,147588,1008,0.692308,0.215615,0.328821


In [79]:
subdf_pred_stats2 = pd.concat([subdf_pred_ling_stats2.T, subdf_pred_pers_stats2.T, subdf_pred_cont_stats2.T])
subdf_pred_stats2

match_type,category_match,exact_match,mismatch
linguistic_count,2,151854,855
personname_count,339,149012,3360
contextual_count,132,148596,3983


Save the data:

In [80]:
subdf_pred_stats.to_csv(config.tokc_path+"model_output/strict_agreement_stats_glove50_crf.csv")
subdf_pred_stats2.to_csv(config.tokc_path+"model_output/match_types_glove50_crf.csv")

### Loose Evaluation

Conduct a loose evaluation of the model's performance (as the manual annotation were evaluated), considering any overlapping or envelopping expected and predicted annotations to be matches, in addition to exactly-matching expected and predicted annotations.

In [81]:
# Get the start and end offsets of the tokens
tok_df = pd.read_csv(config.tokc_path+"desc_sent_ann_token_tag.csv", usecols=["token_id", "sentence_id", "token_offsets"])
tok_df.head()

,token_id,sentence_id,token_offsets
0,0,0,"(0, 10)"
1,1,0,"(10, 11)"
2,2,0,"(12, 15)"
3,3,1,"(17, 22)"
4,4,1,"(22, 23)"


In [82]:
# Join the offsets data to the dev data, keeping only the tokens included in the dev data
df_dev_exploded = df_dev_exploded.reset_index()
join_cols = ["sentence_id", "token_id"]
df_pred = df_dev_exploded.join(tok_df.set_index(join_cols), on=join_cols, how="left")
df_pred = df_pred[["sentence_id", "token_id", "token", "token_offsets", "pos", 
                   "tag_cat_linguistic_expected", "tag_cat_linguistic_predicted",
                   "tag_cat_personname_expected", "tag_cat_personname_predicted",
                   "tag_cat_contextual_expected", "tag_cat_contextual_predicted"
                  ]]
df_pred.head()

,sentence_id,token_id,token,token_offsets,pos,tag_cat_linguistic_expected,tag_cat_linguistic_predicted,tag_cat_personname_expected,tag_cat_personname_predicted,tag_cat_contextual_expected,tag_cat_contextual_predicted
0,5,154,After,"(907, 912)",IN,O,O,O,O,O,O
1,5,155,his,"(913, 916)",PRP$,B-Linguistic,B-Linguistic,O,O,O,O
2,5,156,ordination,"(917, 927)",NN,O,O,O,O,O,O
3,5,157,he,"(928, 930)",PRP,B-Linguistic,B-Linguistic,O,O,O,O
4,5,158,spent,"(931, 936)",VBD,O,O,O,O,O,O


In [83]:
assert df_pred.isna().values.any() == False, "There should be no NaN values in any of the prediction DataFrame's columns."
assert df_pred.shape[0] == df_dev_exploded.shape[0], "There should be the same number of rows as the dev DataFrame prior to the join."

In [84]:
# Transform the offsets to tuples of ints
offsets = list(df_pred.token_offsets)
offsets = [tuple(((offset_pair[1:-1]).split(", "))) for offset_pair in offsets]
offsets = [(int(start_offset), int(end_offset)) for start_offset,end_offset in offsets]
# print(type(offsets[0]), type(offsets[0][0]), type(offsets[0][1]))
col_i = list(df_pred.columns).index("token_offsets")
df_pred = df_pred.drop(columns=["token_offsets"])
df_pred.insert((col_i-1), "token_offsets", offsets)
# df_pred.head()

Determine how many expected **Linguistic** tags overlap, envelop/fall within, or exactly match predicted Linguistic tags:

In [85]:
# LINGUISTIC
ling_tags = ["B-Linguistic", "I-Linguistic"]
b_ling_exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_linguistic_expected", ling_tags)
b_ling_pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_linguistic_predicted", ling_tags)
tp, fp, fn = utils.looseAgreement(b_ling_exp_tree, b_ling_pred_tree)
print(ling_tags)
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

['B-Linguistic', 'I-Linguistic']
---------------------------------------
TP: 3235 | FP: 314 | FN: 482
---------------------------------------
Precision: 0.9115243730628346
Recall: 0.8703255313424805
F_1 Score: 0.8904486650151391


In [86]:
# B-LINGUISTIC
b_ling_exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_linguistic_expected", [ling_tags[0]])
b_ling_pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_linguistic_predicted", [ling_tags[0]])
tp, fp, fn = utils.looseAgreement(b_ling_exp_tree, b_ling_pred_tree)
print(ling_tags[0])
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

B-Linguistic
---------------------------------------
TP: 2854 | FP: 292 | FN: 287
---------------------------------------
Precision: 0.9071837253655436
Recall: 0.9086278255332697
F_1 Score: 0.9079052012088437


In [87]:
# I-LINGUISTIC
b_ling_exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_linguistic_expected", [ling_tags[1]])
b_ling_pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_linguistic_predicted", [ling_tags[1]])
tp, fp, fn = utils.looseAgreement(b_ling_exp_tree, b_ling_pred_tree)
print(ling_tags[1])
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

I-Linguistic
---------------------------------------
TP: 76 | FP: 25 | FN: 201
---------------------------------------
Precision: 0.7524752475247525
Recall: 0.2743682310469314
F_1 Score: 0.4021164021164021


Determine how many expected **Person Name** tags overlap, envelop/fall within, or exactly match predicted Person Name tags:

In [88]:
# PERSON NAME
tags = ["B-Person-Name", "I-Person-Name"]
exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_personname_expected", tags)
pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_personname_predicted", tags)
tp, fp, fn = utils.looseAgreement(exp_tree, pred_tree)
print(tags)
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

['B-Person-Name', 'I-Person-Name']
---------------------------------------
TP: 40830 | FP: 853 | FN: 2104
---------------------------------------
Precision: 0.9795360218794232
Recall: 0.9509945497740718
F_1 Score: 0.9650543035087512


In [89]:
# B-PERSON-NAME
exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_personname_expected", [tags[0]])
pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_personname_predicted", [tags[0]])
tp, fp, fn = utils.looseAgreement(exp_tree, pred_tree)
print(tags[0])
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

B-Person-Name
---------------------------------------
TP: 8864 | FP: 498 | FN: 896
---------------------------------------
Precision: 0.9468062379833369
Recall: 0.9081967213114754
F_1 Score: 0.9270996757661333


In [90]:
# I-PERSON-NAME
exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_personname_expected", [tags[1]])
pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_personname_predicted", [tags[1]])
tp, fp, fn = utils.looseAgreement(exp_tree, pred_tree)
print(tags[1])
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

I-Person-Name
---------------------------------------
TP: 15733 | FP: 675 | FN: 1578
---------------------------------------
Precision: 0.9588615309605071
Recall: 0.9088440875743746
F_1 Score: 0.933183071858596


Determine how many expected **Contextual** tags overlap, envelop/fall within, or exactly match predicted Person Name tags:

In [91]:
# CONTEXTUAL
tags = ["B-Contextual", "I-Contextual"]
exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_contextual_expected", tags)
pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_contextual_predicted", tags)
tp, fp, fn = utils.looseAgreement(exp_tree, pred_tree)
print(tags)
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

['B-Contextual', 'I-Contextual']
---------------------------------------
TP: 6932 | FP: 300 | FN: 3397
---------------------------------------
Precision: 0.9585176991150443
Recall: 0.6711201471584858
F_1 Score: 0.789476681282387


In [92]:
# B-CONTEXTUAL
exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_contextual_expected", [tags[0]])
pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_contextual_predicted", [tags[0]])
tp, fp, fn = utils.looseAgreement(exp_tree, pred_tree)
print(tags[0])
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

B-Contextual
---------------------------------------
TP: 2077 | FP: 95 | FN: 1293
---------------------------------------
Precision: 0.9562615101289135
Recall: 0.6163204747774481
F_1 Score: 0.749548899314327


In [93]:
# I-CONTEXTUAL
exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_contextual_expected", [tags[1]])
pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_contextual_predicted", [tags[1]])
tp, fp, fn = utils.looseAgreement(exp_tree, pred_tree)
print(tags[1])
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

I-Contextual
---------------------------------------
TP: 2117 | FP: 248 | FN: 2347
---------------------------------------
Precision: 0.8951374207188161
Recall: 0.4742383512544803
F_1 Score: 0.6200029286864841
